In [8]:
import requests
from bs4 import BeautifulSoup

def fetch_iframe_title(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.select_one('html body div h2 a span')
    return title.text if title else 'No Title Found'

def process_markdown(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    lines = content.split('\n')
    new_lines = []
    iframe_url = None

    for line in lines:
        line = line.strip()
        if line.startswith('源地址:'):
            iframe_url = line.split('源地址: ')[1]
            title = fetch_iframe_title(iframe_url)
            new_lines.append(f'### {title}')
            continue
        if line.startswith('*爬取时间') or line.startswith('## Iframe') or line.startswith('### Iframe') or line.startswith('#### 表格') or len(line) == 0:
            continue
        new_lines.append(line)

    title_line = next(line for line in lines if line.startswith('# '))
    output_file = title_line[2:].strip() + '.md'

    with open(output_file, 'w', encoding='utf-8') as file:
        file.write('\n'.join(new_lines))


In [ ]:
process_markdown('/Users/ianleely/Documents/Codes/aws-genai-try-it-on/iframe-result/20250307_113025_highspot.md')

In [7]:
import os
from tqdm import tqdm

def list_files_in_directory(directory_path):
    return os.listdir(directory_path)

# 示例用法
directory_path = '/Users/ianleely/Documents/Codes/aws-genai-try-it-on/iframe-result/'
file_names = list_files_in_directory(directory_path)

for file in tqdm(file_names):
    file_path = os.path.join(directory_path, file)
    process_markdown(file_path)

100%|██████████| 7/7 [00:58<00:00,  8.29s/it]


In [3]:
import os
from tqdm import tqdm
import json

def list_files_in_directory(directory_path):
    return os.listdir(directory_path)

# 示例用法
directory_path = '/Users/ianleely/Documents/Codes/aws-genai-try-it-on/iframe-result/'
file_names = list_files_in_directory(directory_path)

for file_name in tqdm(file_names):
    if not file_name.endswith('.md'):
        continue
    row = file_name.replace('.md', '')
    sample = {
        "metadataAttributes": {
            "x-amz-bedrock-kb-source-uri": {
                "value": {
                    "type": "STRING",
                    "stringValue": f"https://aws.highspot.com/{row}"
                },
                "includeForEmbedding": True
            },
            "updated_date": {
                "value": {
                    "type": "NUMBER",
                    "numberValue": 20250307
                },
                "includeForEmbedding": True
            },
            "create_time": {
                "value": {
                    "type": "NUMBER",
                    "numberValue": 20241030
                },
                "includeForEmbedding": True
            },
            "is_partner_solution": {
                "value": {
                    "type": "BOOLEAN",
                    "booleanValue": True
                },
                "includeForEmbedding": True
            }
        }
    }
    json.dump(sample, open(directory_path+row+'.metadata.json', 'w'))

100%|██████████| 9/9 [00:00<00:00, 3112.01it/s]
